In [1]:
#Load libraries 

import pandas as pd
import sspa
import scipy
import numpy as np 
import os #get path location
import pickle

We make two different networks, one for the COVID cases 1-2 compared to COVID cases 3-7 <br>
This is because there are only 18 samples in common between the metabolomic and proteomic datasets

0       Common samples: 18           Metabolomic samples: 133        Proteomic samples: 123 <br>
1-2       Common samples: 45          Metabolomic samples: 45        Proteomic samples: 48 <br>
3-4       Common samples: 56          Metabolomic samples: 57        Proteomic samples: 59 <br>
5-7       Common samples: 27          Metabolomic samples: 28        Proteomic samples: 28 <br>

146 common samples overall,   128 cases (45 samples (WHO 1-2) vs 83 samples (WHO 3-7))

In [6]:
#Load dataset
df = pd.read_csv('Data/Su_COVID_proteomics_processed.csv', index_col=0)

df2 = pd.read_csv('Data/Su_COVID_metabolomics_processed_ChEBI.csv', index_col=0)
df2.index= df2.index.str.rstrip('-BL')


intersection = list(set(df.index.tolist()) & set(df2.index.tolist())) #set removes duplicates
intersection = [sample for sample in intersection if sample.startswith("INCOV")]
print(len(intersection))

df = df[df.index.isin(intersection)]

df_mild = (df[df["WHO_status"] == '1-2']).iloc[:,:-2] #45 samples, remove the metadata
df_severe = (df[(df["WHO_status"] == '3-4') | (df["WHO_status"] == '5-7')]).iloc[:,:-2] #83 samples

128


In [3]:
#Download the reactome pathways
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_all_Levels.txt', download_latest = False, filepath = None)

#Download the root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())



### Step 1: Determine initial test-statistic

In [8]:
#Function to calculate the squared Spearman correlation matrix 

def squared_spearman_corr(data):
    kpca_scores = sspa.sspa_kpca(data, reactome_pathways)   
    kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns))) #using Sara's code to drop root pathways

    spearman_results = scipy.stats.spearmanr(kpca_scores)
    squared_spearman_coef = np.square(spearman_results[0]) #correlation coefficients (spearman_results[1] gives the p-values)

    return squared_spearman_coef,list(kpca_scores.columns)




#Function to calculate the absolute difference between two matrices and then determine the mean for each edge

def absolute_val(data1,data2,edgelist):
    abs_rho_squared = np.absolute(np.array(data1) - np.array(data2))

    #Mask the upper half of the dataframe (so I don't view the comparisons between the two same genes, and also the duplicate comparisons are removed)
    mask = abs_rho_squared.copy()
    mask = np.triu(np.ones(mask.shape)).astype(bool)
    mask = np.invert(mask) #invert true and false values so the diagonal is False as well
    non_dup_abs_squared = pd.DataFrame(abs_rho_squared, columns = edgelist, index = edgelist)
    non_dup_abs_squared = pd.DataFrame(non_dup_abs_squared).where(mask) #Replace all false values with NaN using mask

    abs_squared_list = non_dup_abs_squared.stack().reset_index()
    abs_squared_list['level_0'] = abs_squared_list["level_0"].astype(str) + ", " + abs_squared_list['level_1']
    abs_squared_list.columns = ["Edges","na","Initial_tstat"]
    abs_squared_list.index = abs_squared_list["Edges"]
    abs_squared_list = abs_squared_list.drop(columns = ["Edges","na"])

    return(abs_squared_list)

Note: For the delta squared correlation values for the unshuffled data (i.e. the real data) I keep the indices (pathway edges). Since I already have a record of the edges, there is no need to keep the edges for each permutation, since the order is the same each time. 

In [9]:
spearman_mild,edgelist = squared_spearman_corr(df_mild)
spearman_severe,edgelist = squared_spearman_corr(df_severe)

output = absolute_val(spearman_mild,spearman_severe,edgelist)

In [11]:
output.to_csv("Data/permutation_test_files__proteomics/initial_tstats.csv")

### Step 2: Shuffle the labels

The sample labels are shuffled, rather than assigning the samples to two different groups (since the sizes of the 1-2 class with the 3-7 class is not equal). See HPC_permutation_script_metabolomic for more info.

### Step 3: Read in the permutation files 

In [2]:
#Get location of the script and append the path to the folder that the test files are stored in
path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'

final_list = []

#Unpickle the permutation test files and append as a list as a list of lists
for filename in os.listdir(path):
   if filename != "initial_tstats.csv":
      with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
         list1 = pickle.load(f)
      final_list.append(list1)

In [3]:
for list1 in final_list:
    print(list1[:15])
    #print(list1[0])
    #print(len(list1))

[0.24337239352125428, -0.011366924360905563, 0.22336974083843036, 0.21587802738661835, 0.04949002003751546, 0.17928142937837468, -0.10142569491204587, 0.02257251541271951, 0.08779882397402469, 0.03401564629193614, 0.0143398248868673, -0.5409505660211041, -0.040493060535651056, -0.490443425841589, 0.0017368291941227405]
[-0.07712829336327946, -0.014813301075029213, -0.023159472559369218, -0.14738425307625702, -0.1570943825881833, 0.0398722477420983, 0.027863440419200344, -0.0009759726418205286, -0.09606479669900453, -0.00705925661009887, -0.035959242571532934, -0.08109864504703046, -0.016535233121105095, -0.08713570292976403, -0.003340580932105594]
[-0.20153733476360788, -0.059125113507662995, 0.22031714680168507, -0.2326411715202603, 0.04158696037794796, 0.26219381255839197, -0.12884582765365607, 0.041124510761897395, 0.14699690745822788, 0.0390670743950656, -0.07398292470389146, -0.15244310216361856, 0.0010123196360730619, -0.17391844258356054, -0.014808837899154059]
[-0.0177039767868

In [5]:
df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)

https://www.jwilber.me/permutationtest/# <br>
We take the absolute value for the absolute difference between the two groups to compute the initial test statistic. Each permutation calculates a new test statistic calculated from the ABSOLUTE difference between the two test groups. It makes no difference whether I take the absolute difference or regular difference between the two, since either way it's a two-tailed p-value distribution, just that with the absolute difference the negative values have been mapped to the positive side.

In [6]:
sig_edges = []
edgelist = df.index 

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    #print("comparison: ",comparison)
    #print(pathway_pair)
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   #two-tailed permutation test means we test on both sides  
            counter += 1    #if the permutation test statistic is over the initial test statistic, add to counter
    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 tests, p_val of 0.01 means that 1000 permutations have a test statistic higher than the initial one
        sig_edges.append(pathway_pair)  

len(sig_edges)

1316

In [ ]:

len(sig_edges)
sig_edges

In [ ]:
with open("Data/sig_edges.txt", "wb") as file_output:   #Pickling (stores as list format instead of string, easier to read in later)
      pickle.dump(sig_edges , file_output)

### Step 4: Compare the difference in edges with the naive networks (mild vs severe) to see if they agree with the differential network results

In [1]:
import networkx as nx

In [2]:
mild = nx.read_gml("Cytoscape/proteomic_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/proteomic_final_severecases.gml")

In [3]:
mild_edges = list(mild.edges())
print(len(mild_edges))
severe_edges = list(severe.edges())
print(len(severe_edges))

8061
67986


In [4]:
#Edges present in the mild network but not the severe network
len(set(mild_edges) - set(severe_edges))

4357

In [5]:
#Edges present in the severe network but not the mild network
len(set(severe_edges) - set(mild_edges))

64282

In [6]:
#Reading in the edges expressed in the differential metabolomic network
with open('Data/permutation_test_files_proteomics/sig_edges.txt') as f:
    lines = f.readlines()

edges_remaining = []

edges = lines[0].split(",")

for index in range(0,len(edges),2):
    list1 = edges[index],(edges[index+1][1:])
    edges_remaining.append(tuple(list1))

print(len(edges_remaining))
edges_remaining

29327


[('R-HSA-111465', 'R-HSA-109581'),
 ('R-HSA-112314', 'R-HSA-109581'),
 ('R-HSA-112314', 'R-HSA-109606'),
 ('R-HSA-112315', 'R-HSA-109581'),
 ('R-HSA-114508', 'R-HSA-109581'),
 ('R-HSA-1168372', 'R-HSA-111465'),
 ('R-HSA-1168372', 'R-HSA-112314'),
 ('R-HSA-1168372', 'R-HSA-112315'),
 ('R-HSA-1168372', 'R-HSA-114508'),
 ('R-HSA-1169091', 'R-HSA-109606'),
 ('R-HSA-1169091', 'R-HSA-111465'),
 ('R-HSA-1169091', 'R-HSA-112314'),
 ('R-HSA-1169091', 'R-HSA-112315'),
 ('R-HSA-1169091', 'R-HSA-114508'),
 ('R-HSA-1226099', 'R-HSA-109581'),
 ('R-HSA-1226099', 'R-HSA-109606'),
 ('R-HSA-1226099', 'R-HSA-1168372'),
 ('R-HSA-1226099', 'R-HSA-1169091'),
 ('R-HSA-1227986', 'R-HSA-112314'),
 ('R-HSA-1227986', 'R-HSA-112315'),
 ('R-HSA-1227986', 'R-HSA-114508'),
 ('R-HSA-1227986', 'R-HSA-1169091'),
 ('R-HSA-1227990', 'R-HSA-112314'),
 ('R-HSA-1227990', 'R-HSA-112315'),
 ('R-HSA-1227990', 'R-HSA-114508'),
 ('R-HSA-1227990', 'R-HSA-1169091'),
 ('R-HSA-1234174', 'R-HSA-109581'),
 ('R-HSA-1234176', 'R-HSA-109

In [7]:
intersection = list(set(edges_remaining).intersection(list(set(mild_edges))))  
len(intersection) #659 edges out of 4,357

659

In [8]:
intersection = list(set(edges_remaining).intersection(list(set(severe_edges)))) 
len(intersection) #2517 edges out of 64,282

2517